<a href="https://colab.research.google.com/github/dragonslayer551/StudeoMea/blob/master/Backend-Spotify-ML-Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spotipy

In [ ]:
pip install --upgrade scikit-learn

     |████████████████████████████████| 22.2MB 65.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import csv

cid = '810fa38e52f84cb79d7d2bd367810c2f'
secret = '6b590eab445149ae89b05ae48577e460'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

def get_features(track_id):
    features_results = sp.audio_features([track_id])
    json_features = json.dumps(features_results)
    features_data = json.loads(json_features)

    # Convert features dictionary to a list
    features_list = list(features_data[0].values())

    return features_list

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# IDs of monthly playlists from November 2016 to November 2017
playlist_ids = [
    "2p0AaZrfASsiPJBTEFYOTH",
    "0yzaEM4q8f0cSK6wYRa4IT",
    "3KnSzUcuUZDui7HUPh1r50"
]

# Audio features
feature_names = [
  "danceability",
  "energy",
  "key",
  "loudness",
  "mode",
  "speechiness",
  "acousticness",
  "instrumentalness",
  "liveness",
  "valence",
  "tempo",
  "type",
  "id",
  "uri",
  "track_href",
  "analysis_url",
  "duration_ms",
  "time_signature"
]

username = 'SacHacks'

### Write data to CSV file
data_file = open('data.csv','w')
writer = csv.writer(data_file)

# Write header
writer.writerow(['track_id', 'playlist_id', 'date_added', 'track_name', 'first_artist'] + feature_names + ['lyrics', 'neg', 'neu', 'pos', 'compound'])

for playlist_id in playlist_ids:
    print('Querying playlist: ' + str(playlist_id))

    repeat_query = True
    offset_n = 0
    for i in range(2):
        # Query Spotify API
        if i > 0:
            print('Repeating query')
            offset_n += 100
        results = sp.user_playlist_tracks(username, playlist_id, offset=offset_n)
        json_results = json.dumps(results)
        data = json.loads(json_results)

        # Write rows
        for track in data['items']:
            track_id = track['track']['id']
            date_added = track['added_at']
            track_name = track['track']['name']
            first_artist = track['track']['artists'][0]['name']

            # Track features
            features = get_features(track_id)

            # Try to get lyrics, if available
            lyrics = ''
            try:
                lyrics = pylyrics3.get_song_lyrics(first_artist, track_name)
            except:
                pass

            # Sentiment Analysis
            neg = None
            neu = None
            pos = None
            compound = None
            if lyrics:
                snt = sentiment_analyzer.polarity_scores(lyrics)
                neg = snt['neg']
                neu = snt['neu']
                pos = snt['pos']
                compound = snt['compound']

            writer.writerow([track_id, playlist_id, date_added, track_name, first_artist] + features + [lyrics] + [neg, neu, pos, compound])

        # Special case: API limit is 100 tracks, so we need a second request
        # for playlists that have over 100 tracks
        if data['total'] < 100:
            break

    print('Done querying')

data_file.close()

Querying playlist: 2p0AaZrfASsiPJBTEFYOTH
Done querying
Querying playlist: 0yzaEM4q8f0cSK6wYRa4IT
Done querying
Querying playlist: 3KnSzUcuUZDui7HUPh1r50
Done querying


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
data.head()

,track_id,playlist_id,date_added,track_name,first_artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,lyrics,neg,neu,pos,compound
0,0VNzEY1G4GLqcNx5qaaTl6,2p0AaZrfASsiPJBTEFYOTH,2020-12-16T06:29:30Z,Vincent,Don McLean,0.419,0.085,7,-19.387,1,0.0362,0.9080,0.000005,0.3400,0.458,91.509,audio_features,0VNzEY1G4GLqcNx5qaaTl6,spotify:track:0VNzEY1G4GLqcNx5qaaTl6,https://api.spotify.com/v1/tracks/0VNzEY1G4GLq...,https://api.spotify.com/v1/audio-analysis/0VNz...,243493,3,NaN,NaN,NaN,NaN,NaN
1,0ll4shInilQVdYVrVVHpjB,2p0AaZrfASsiPJBTEFYOTH,2020-12-16T06:34:41Z,Soldier of Love,Pearl Jam,0.550,0.777,4,-7.299,1,0.0382,0.1430,0.000007,0.9810,0.751,116.140,audio_features,0ll4shInilQVdYVrVVHpjB,spotify:track:0ll4shInilQVdYVrVVHpjB,https://api.spotify.com/v1/tracks/0ll4shInilQV...,https://api.spotify.com/v1/audio-analysis/0ll4...,174173,4,NaN,NaN,NaN,NaN,NaN
2,4U45aEWtQhrm8A5mxPaFZ7,2p0AaZrfASsiPJBTEFYOTH,2020-12-16T07:00:08Z,Vienna,Billy Joel,0.532,0.495,10,-6.662,1,0.0343,0.6590,0.000000,0.0754,0.308,124.936,audio_features,4U45aEWtQhrm8A5mxPaFZ7,spotify:track:4U45aEWtQhrm8A5mxPaFZ7,https://api.spotify.com/v1/tracks/4U45aEWtQhrm...,https://api.spotify.com/v1/audio-analysis/4U45...,214240,4,NaN,NaN,NaN,NaN,NaN
3,0EhYGqH7kjogumPfbKWh0t,2p0AaZrfASsiPJBTEFYOTH,2020-12-16T07:14:49Z,Cecily Smith (Bonus Track),Will Connolly,0.771,0.263,7,-13.072,1,0.0816,0.8210,0.000000,0.0872,0.372,119.726,audio_features,0EhYGqH7kjogumPfbKWh0t,spotify:track:0EhYGqH7kjogumPfbKWh0t,https://api.spotify.com/v1/tracks/0EhYGqH7kjog...,https://api.spotify.com/v1/audio-analysis/0EhY...,289893,4,NaN,NaN,NaN,NaN,NaN
4,3VygfAvvgVaJUeaBUSzlZu,2p0AaZrfASsiPJBTEFYOTH,2020-12-16T07:19:15Z,Karma,AJR,0.528,0.727,6,-5.739,1,0.1790,0.0557,0.000000,0.1760,0.212,155.927,audio_features,3VygfAvvgVaJUeaBUSzlZu,spotify:track:3VygfAvvgVaJUeaBUSzlZu,https://api.spotify.com/v1/tracks/3VygfAvvgVaJ...,https://api.spotify.com/v1/audio-analysis/3Vyg...,245270,4,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,neg,neu,pos,compound
count,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,111.000000,0.0,0.0,0.0,0.0,0.0
mean,0.553883,0.509382,5.513514,-9.361495,0.792793,0.070190,0.417091,0.165283,0.195899,0.451947,121.988486,205176.747748,3.873874,NaN,NaN,NaN,NaN,NaN
std,0.169601,0.239019,3.564637,5.276788,0.407143,0.070275,0.357015,0.336248,0.178197,0.203079,27.352984,79965.331084,0.384167,NaN,NaN,NaN,NaN,NaN
min,0.137000,0.009650,0.000000,-34.079000,0.000000,0.026100,0.000021,0.000000,0.042900,0.038200,72.684000,58869.000000,3.000000,NaN,NaN,NaN,NaN,NaN
25%,0.429000,0.352000,2.000000,-10.986000,1.000000,0.035950,0.050850,0.000000,0.105000,0.291500,102.444500,164922.500000,4.000000,NaN,NaN,NaN,NaN,NaN
50%,0.560000,0.487000,6.000000,-8.165000,1.000000,0.046100,0.379000,0.000087,0.119000,0.452000,119.864000,201264.000000,4.000000,NaN,NaN,NaN,NaN,NaN
75%,0.671000,0.688000,9.000000,-5.773500,1.000000,0.061200,0.749000,0.025200,0.220500,0.570000,136.970500,227785.000000,4.000000,NaN,NaN,NaN,NaN,NaN
max,0.918000,0.987000,11.000000,-2.770000,1.000000,0.424000,0.995000,0.961000,0.981000,0.943000,186.142000,798693.000000,5.000000,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.columns.values

array(['track_id', 'playlist_id', 'date_added', 'track_name',
       'first_artist', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href',
       'analysis_url', 'duration_ms', 'time_signature', 'lyrics', 'neg',
       'neu', 'pos', 'compound'], dtype=object)

In [ ]:
data.drop_duplicates(subset=['track_id'], keep='first', inplace=True)

In [ ]:
data.dtypes

track_id             object
playlist_id          object
date_added           object
track_name           object
first_artist         object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
type                 object
id                   object
uri                  object
track_href           object
analysis_url         object
duration_ms           int64
time_signature        int64
lyrics              float64
neg                 float64
neu                 float64
pos                 float64
compound            float64
dtype: object

In [ ]:
# Drop unused columns
data.drop(['playlist_id', 'date_added', 'track_name', 'first_artist'], axis=1, inplace=True)


In [ ]:
data.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,lyrics,neg,neu,pos,compound
0,0VNzEY1G4GLqcNx5qaaTl6,0.419,0.085,7,-19.387,1,0.0362,0.9080,0.000005,0.3400,0.458,91.509,audio_features,0VNzEY1G4GLqcNx5qaaTl6,spotify:track:0VNzEY1G4GLqcNx5qaaTl6,https://api.spotify.com/v1/tracks/0VNzEY1G4GLq...,https://api.spotify.com/v1/audio-analysis/0VNz...,243493,3,NaN,NaN,NaN,NaN,NaN
1,0ll4shInilQVdYVrVVHpjB,0.550,0.777,4,-7.299,1,0.0382,0.1430,0.000007,0.9810,0.751,116.140,audio_features,0ll4shInilQVdYVrVVHpjB,spotify:track:0ll4shInilQVdYVrVVHpjB,https://api.spotify.com/v1/tracks/0ll4shInilQV...,https://api.spotify.com/v1/audio-analysis/0ll4...,174173,4,NaN,NaN,NaN,NaN,NaN
2,4U45aEWtQhrm8A5mxPaFZ7,0.532,0.495,10,-6.662,1,0.0343,0.6590,0.000000,0.0754,0.308,124.936,audio_features,4U45aEWtQhrm8A5mxPaFZ7,spotify:track:4U45aEWtQhrm8A5mxPaFZ7,https://api.spotify.com/v1/tracks/4U45aEWtQhrm...,https://api.spotify.com/v1/audio-analysis/4U45...,214240,4,NaN,NaN,NaN,NaN,NaN
3,0EhYGqH7kjogumPfbKWh0t,0.771,0.263,7,-13.072,1,0.0816,0.8210,0.000000,0.0872,0.372,119.726,audio_features,0EhYGqH7kjogumPfbKWh0t,spotify:track:0EhYGqH7kjogumPfbKWh0t,https://api.spotify.com/v1/tracks/0EhYGqH7kjog...,https://api.spotify.com/v1/audio-analysis/0EhY...,289893,4,NaN,NaN,NaN,NaN,NaN
4,3VygfAvvgVaJUeaBUSzlZu,0.528,0.727,6,-5.739,1,0.1790,0.0557,0.000000,0.1760,0.212,155.927,audio_features,3VygfAvvgVaJUeaBUSzlZu,spotify:track:3VygfAvvgVaJUeaBUSzlZu,https://api.spotify.com/v1/tracks/3VygfAvvgVaJ...,https://api.spotify.com/v1/audio-analysis/3Vyg...,245270,4,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.corr()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,neg,neu,pos,compound
danceability,1.000000,-0.051154,0.009288,0.168192,-0.177772,0.107736,0.000750,-0.032677,-0.161438,0.393375,-0.216356,-0.327948,0.168270,NaN,NaN,NaN,NaN,NaN
energy,-0.051154,1.000000,0.037102,0.784049,-0.018609,0.129261,-0.749370,-0.504020,0.101778,0.378513,0.158677,0.188677,0.386842,NaN,NaN,NaN,NaN,NaN
key,0.009288,0.037102,1.000000,0.043729,-0.113038,-0.002728,-0.017867,0.100701,-0.086081,-0.026090,-0.055801,-0.035822,0.057661,NaN,NaN,NaN,NaN,NaN
loudness,0.168192,0.784049,0.043729,1.000000,-0.061973,0.112662,-0.670363,-0.653549,0.109775,0.398574,0.095129,0.133622,0.332726,NaN,NaN,NaN,NaN,NaN
mode,-0.177772,-0.018609,-0.113038,-0.061973,1.000000,-0.197966,0.010519,-0.092314,0.019012,-0.018061,-0.000045,0.064031,0.011838,NaN,NaN,NaN,NaN,NaN
speechiness,0.107736,0.129261,-0.002728,0.112662,-0.197966,1.000000,-0.060444,-0.111781,0.215581,0.266486,-0.035001,-0.251657,0.176325,NaN,NaN,NaN,NaN,NaN
acousticness,0.000750,-0.749370,-0.017867,-0.670363,0.010519,-0.060444,1.000000,0.430786,-0.079062,-0.353428,-0.223571,-0.249598,-0.250205,NaN,NaN,NaN,NaN,NaN
instrumentalness,-0.032677,-0.504020,0.100701,-0.653549,-0.092314,-0.111781,0.430786,1.000000,-0.191419,-0.278646,-0.082357,-0.119770,-0.117302,NaN,NaN,NaN,NaN,NaN
liveness,-0.161438,0.101778,-0.086081,0.109775,0.019012,0.215581,-0.079062,-0.191419,1.000000,0.078179,0.002054,0.132037,0.107630,NaN,NaN,NaN,NaN,NaN
valence,0.393375,0.378513,-0.026090,0.398574,-0.018061,0.266486,-0.353428,-0.278646,0.078179,1.000000,-0.014463,-0.240057,0.173900,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,neg,neu,pos,compound
count,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,0.0,0.0,0.0,0.0,0.0
mean,0.556282,0.505295,5.472727,-9.415418,0.800000,0.070274,0.420882,0.166785,0.194816,0.449237,121.614973,204971.218182,3.872727,NaN,NaN,NaN,NaN,NaN
std,0.168475,0.236184,3.554837,5.270127,0.401831,0.070591,0.356397,0.337412,0.178646,0.201983,27.192312,80301.849097,0.385734,NaN,NaN,NaN,NaN,NaN
min,0.137000,0.009650,0.000000,-34.079000,0.000000,0.026100,0.000021,0.000000,0.042900,0.038200,72.684000,58869.000000,3.000000,NaN,NaN,NaN,NaN,NaN
25%,0.442750,0.351000,2.000000,-10.998000,1.000000,0.035875,0.053600,0.000000,0.105000,0.290750,102.214250,164543.750000,4.000000,NaN,NaN,NaN,NaN,NaN
50%,0.561500,0.485500,6.000000,-8.232000,1.000000,0.046000,0.383500,0.000090,0.119000,0.451500,119.795000,201212.000000,4.000000,NaN,NaN,NaN,NaN,NaN
75%,0.671000,0.677500,8.750000,-5.818000,1.000000,0.061300,0.750000,0.025600,0.217750,0.566500,135.524750,227379.500000,4.000000,NaN,NaN,NaN,NaN,NaN
max,0.918000,0.987000,11.000000,-2.770000,1.000000,0.424000,0.995000,0.961000,0.981000,0.943000,186.142000,798693.000000,5.000000,NaN,NaN,NaN,NaN,NaN
